In [1]:
import sys
sys.path.append("..")

In [2]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from langchain.agents import Tool

from tools.mercantile_tool import MercantileTool
from agents.l4m_agent import base_agent

In [3]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [4]:
# pick a LLM
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo"
)

In [5]:
# define a set of tools the agent has access to for queries
duckduckgo_tool = Tool(
    name="DuckDuckGo",
    description="Use this tool to answer questions about current events and places. \
    Please ask targeted questions.",
    func=DuckDuckGoSearchRun().run
)

mercantile_tool = MercantileTool()

tools = [duckduckgo_tool, mercantile_tool]

In [6]:
agent = base_agent(llm, tools) 

In [8]:
agent("What is the xyz tile for London?")



> Entering new AgentExecutor chain...
I need to use mercantile to get the xyz tile for London, but I need to find the longitude and latitude coordinates for London first.
Action: DuckDuckGo
Action Input: "London longitude and latitude"
Observation: London, city, capital of the United Kingdom. It is among the oldest of the world's great cities—its history spanning nearly two millennia—and one of the most cosmopolitan. By far Britain's largest metropolis, it is also the country's economic, transportation, and cultural centre. London is situated in southeastern England, lying astride the River Thames some 50 miles (80 km) upstream ... The length of a degree of arc of latitude is approximately 111 km (69 miles), varying, because of the nonuniformity of Earth's curvature, from 110.567 km (68.706 miles) at the Equator to 111.699 km (69.41 miles) at the poles. Geographic latitude is also given in degrees, minutes, and seconds. facts about lines of longitude London (51°30′N) is farther north

{'input': 'What is the xyz tile for London?',
 'output': 'Tile(x=511, y=340, z=10.0)'}